In [1]:
# Call Center Queueing Simulation Analysis
# Mata Kuliah Pemodelan dan Simulasi Data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta

# Create directory for saving results
if not os.path.exists('ImageStep1'):
    os.makedirs('ImageStep1')

# Step 1: Load and Explore the Dataset
print("Step 1: Loading and Exploring the Dataset")

# Load the dataset
df = pd.read_csv('simulated_call_centre.csv')

# Display basic information
print("\nDataset Information:")
print(f"Total number of records: {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print("\nFirst 5 rows:")
print(df.head())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Convert time columns to datetime format
for col in ['call_started', 'call_answered', 'call_ended']:
    df[col] = pd.to_datetime(df['date'] + ' ' + df[col])

# Basic statistics
print("\nBasic statistics for wait and service times:")
print(df[['wait_length', 'service_length']].describe())

# Calculate additional metrics for exploration
df['total_time'] = df['wait_length'] + df['service_length']
print("\nTotal time statistics (wait + service):")
print(df['total_time'].describe())

# Count calls that meet standard
standard_met_count = df['meets_standard'].value_counts()
print("\nCalls meeting standard:")
print(standard_met_count)

# Create time-based features
df['hour'] = df['call_started'].dt.hour
df['weekday'] = df['call_started'].dt.day_name()

# Visualize call distribution by hour
plt.figure(figsize=(10, 6))
call_by_hour = df['hour'].value_counts().sort_index()
sns.barplot(x=call_by_hour.index, y=call_by_hour.values)
plt.title('Call Distribution by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Calls')
plt.savefig('ImageStep1/call_distribution_by_hour.png')
plt.close()

# Visualize service time distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['service_length'], bins=30, kde=True)
plt.title('Service Time Distribution')
plt.xlabel('Service Time (seconds)')
plt.ylabel('Frequency')
plt.savefig('ImageStep1/service_time_distribution.png')
plt.close()

Step 1: Loading and Exploring the Dataset

Dataset Information:
Total number of records: 51708
Date range: 2021-01-01 to 2021-12-31

First 5 rows:
   call_id        date  daily_caller call_started call_answered  call_ended  \
0        1  2021-01-01             1   8:00:00 AM    8:00:00 AM  8:14:22 AM   
1        2  2021-01-01             2   8:02:42 AM    8:02:42 AM  8:07:31 AM   
2        3  2021-01-01             3   8:08:24 AM    8:08:24 AM  8:10:13 AM   
3        4  2021-01-01             4   8:09:37 AM    8:09:37 AM  8:13:45 AM   
4        5  2021-01-01             5   8:11:10 AM    8:11:10 AM  8:15:28 AM   

   wait_length  service_length  meets_standard  
0            0             863            True  
1            0             289            True  
2            0             108            True  
3            0             247            True  
4            0             258            True  

Missing values:
call_id           0
date              0
daily_caller      0
call_st

In [2]:
# Step 2: Implement the Queueing Simulation
print("\nStep 2: Implementing the Queueing Simulation")

def simulate_queue(arrivals, service_times, num_agents):
    """
    Simulates a multi-server queue system
    
    Parameters:
    - arrivals: list of arrival times (in seconds from start)
    - service_times: list of service durations
    - num_agents: number of service agents
    
    Returns:
    - wait_times: waiting times for each customer
    - queue_length: queue length at each arrival time
    - agent_utilization: percentage of time agents are busy
    """
    n = len(arrivals)
    wait_times = np.zeros(n)
    queue_length = np.zeros(n)
    
    # Initialize agent completion times
    agent_busy_until = np.zeros(num_agents)
    total_busy_time = np.zeros(num_agents)
    
    for i in range(n):
        # Find the earliest available agent
        earliest_available_agent = np.argmin(agent_busy_until)
        
        # If agent is available immediately
        if agent_busy_until[earliest_available_agent] <= arrivals[i]:
            wait_times[i] = 0
            agent_busy_until[earliest_available_agent] = arrivals[i] + service_times[i]
        else:
            # Customer must wait
            wait_times[i] = agent_busy_until[earliest_available_agent] - arrivals[i]
            agent_busy_until[earliest_available_agent] += service_times[i]
        
        # Calculate queue length at arrival (customers waiting, not being served)
        queue_length[i] = sum(1 for t in agent_busy_until if t > arrivals[i])
        
        # Update total busy time for the agent
        total_busy_time[earliest_available_agent] += service_times[i]
    
    # Calculate agent utilization
    simulation_end = max(agent_busy_until)
    simulation_duration = simulation_end - min(arrivals)
    agent_utilization = sum(total_busy_time) / (simulation_duration * num_agents) if simulation_duration > 0 else 0
    
    return wait_times, queue_length, agent_utilization

# Prepare data for simulation
# We'll organize the data by date
dates = df['date'].unique()

# For demonstration, let's run the simulation for the first date
print(f"\nRunning simulation for date: {dates[0]}")

# Filter data for the first date
day_data = df[df['date'] == dates[0]].copy()
day_data = day_data.sort_values('call_started')

# Convert timestamps to seconds from start of day
day_start = day_data['call_started'].min()
day_data['arrival_seconds'] = (day_data['call_started'] - day_start).dt.total_seconds()

# Run simulation with different numbers of agents
agent_counts = range(1, 6)  # Test 1 to 5 agents
results = {}

for num_agents in agent_counts:
    wait_times, queue_lengths, utilization = simulate_queue(
        day_data['arrival_seconds'].values,
        day_data['service_length'].values,
        num_agents
    )
    
    results[num_agents] = {
        'wait_times': wait_times,
        'avg_wait': np.mean(wait_times),
        'max_wait': np.max(wait_times),
        'queue_lengths': queue_lengths,
        'avg_queue': np.mean(queue_lengths),
        'max_queue': np.max(queue_lengths),
        'utilization': utilization
    }
    
    print(f"\nResults for {num_agents} agents:")
    print(f"Average wait time: {results[num_agents]['avg_wait']:.2f} seconds")
    print(f"Maximum wait time: {results[num_agents]['max_wait']:.2f} seconds")
    print(f"Average queue length: {results[num_agents]['avg_queue']:.2f}")
    print(f"Maximum queue length: {results[num_agents]['max_queue']:.2f}")
    print(f"Agent utilization: {results[num_agents]['utilization']*100:.2f}%")


Step 2: Implementing the Queueing Simulation

Running simulation for date: 2021-01-01

Results for 1 agents:
Average wait time: 6301.77 seconds
Maximum wait time: 9764.00 seconds
Average queue length: 1.00
Maximum queue length: 1.00
Agent utilization: 100.00%

Results for 2 agents:
Average wait time: 100.84 seconds
Maximum wait time: 1121.00 seconds
Average queue length: 1.75
Maximum queue length: 2.00
Agent utilization: 61.85%

Results for 3 agents:
Average wait time: 6.32 seconds
Maximum wait time: 211.00 seconds
Average queue length: 2.05
Maximum queue length: 3.00
Agent utilization: 41.24%

Results for 4 agents:
Average wait time: 0.50 seconds
Maximum wait time: 73.00 seconds
Average queue length: 2.13
Maximum queue length: 4.00
Agent utilization: 30.93%

Results for 5 agents:
Average wait time: 0.00 seconds
Maximum wait time: 0.00 seconds
Average queue length: 2.14
Maximum queue length: 5.00
Agent utilization: 24.74%


In [3]:
# Create the directory if it doesn't exist
os.makedirs('ImageStep3', exist_ok=True)

print("\nStep 3: Performance Analysis & Visualization")

# Plot wait times by number of agents
plt.figure(figsize=(10, 6))
avg_waits = [results[n]['avg_wait'] for n in agent_counts]
max_waits = [results[n]['max_wait'] for n in agent_counts]

plt.plot(agent_counts, avg_waits, marker='o', label='Average Wait Time')
plt.plot(agent_counts, max_waits, marker='s', label='Maximum Wait Time')
plt.title('Wait Times by Number of Agents')
plt.xlabel('Number of Agents')
plt.ylabel('Wait Time (seconds)')
plt.legend()
plt.grid(True)
plt.savefig('ImageStep3/wait_times_by_agents.png')
plt.close()

# Plot queue lengths by number of agents
plt.figure(figsize=(10, 6))
avg_queues = [results[n]['avg_queue'] for n in agent_counts]
max_queues = [results[n]['max_queue'] for n in agent_counts]

plt.plot(agent_counts, avg_queues, marker='o', label='Average Queue Length')
plt.plot(agent_counts, max_queues, marker='s', label='Maximum Queue Length')
plt.title('Queue Lengths by Number of Agents')
plt.xlabel('Number of Agents')
plt.ylabel('Number of Callers in Queue')
plt.legend()
plt.grid(True)
plt.savefig('ImageStep3/queue_lengths_by_agents.png')
plt.close()

# Plot agent utilization
plt.figure(figsize=(10, 6))
utilizations = [results[n]['utilization']*100 for n in agent_counts]

plt.plot(agent_counts, utilizations, marker='o')
plt.title('Agent Utilization by Number of Agents')
plt.xlabel('Number of Agents')
plt.ylabel('Utilization (%)')
plt.grid(True)
plt.savefig('ImageStep3/agent_utilization.png')
plt.close()


Step 3: Performance Analysis & Visualization


In [4]:
# Step 4: Modify Parameters and Optimize
print("\nStep 4: Modifying Parameters and Optimizing")

# Create the directory if it doesn't exist
import os
os.makedirs('ImageStep4', exist_ok=True)

# Let's find the optimal number of agents based on desired service level
# For example, we might want 95% of calls to have wait times < 60 seconds

# Function to calculate service level
def calculate_service_level(wait_times, threshold=60):
    calls_within_threshold = sum(1 for w in wait_times if w <= threshold)
    return calls_within_threshold / len(wait_times) if len(wait_times) > 0 else 0

# Calculate service level for each agent count
service_levels = {}
thresholds = [30, 60, 120]  # Wait time thresholds in seconds

for threshold in thresholds:
    service_levels[threshold] = []
    for num_agents in agent_counts:
        service_level = calculate_service_level(results[num_agents]['wait_times'], threshold)
        service_levels[threshold].append(service_level * 100)  # Convert to percentage

# Plot service levels
plt.figure(figsize=(12, 6))
for threshold in thresholds:
    plt.plot(agent_counts, service_levels[threshold], marker='o', label=f'Within {threshold}s')

plt.axhline(y=95, color='r', linestyle='--', label='95% Target')
plt.title('Service Level by Number of Agents')
plt.xlabel('Number of Agents')
plt.ylabel('Service Level (%)')
plt.legend()
plt.grid(True)
plt.savefig('ImageStep4/service_levels.png')  # Changed from ImageResult to ImageStep4
plt.close()

# Find optimal number of agents for different criteria
print("\nFinding optimal staffing levels:")

# For 95% service level within 60 seconds
for num_agents in agent_counts:
    service_level_60 = calculate_service_level(results[num_agents]['wait_times'], 60)
    if service_level_60 >= 0.95:
        print(f"For 95% of calls within 60 seconds: {num_agents} agents (actual: {service_level_60*100:.2f}%)")
        break
else:
    print("Could not achieve 95% service level within 60 seconds with the tested agent counts")

# For 90% service level within 30 seconds
for num_agents in agent_counts:
    service_level_30 = calculate_service_level(results[num_agents]['wait_times'], 30)
    if service_level_30 >= 0.90:
        print(f"For 90% of calls within 30 seconds: {num_agents} agents (actual: {service_level_30*100:.2f}%)")
        break
else:
    print("Could not achieve 90% service level within 30 seconds with the tested agent counts")

# Find balance between utilization and wait times
# Higher utilization means fewer idle agents but possibly longer wait times
print("\nBalancing utilization and service level:")
for num_agents in agent_counts:
    utilization = results[num_agents]['utilization'] * 100
    service_level_60 = calculate_service_level(results[num_agents]['wait_times'], 60) * 100
    print(f"{num_agents} agents: {utilization:.2f}% utilization, {service_level_60:.2f}% calls within 60s")


Step 4: Modifying Parameters and Optimizing

Finding optimal staffing levels:
For 95% of calls within 60 seconds: 3 agents (actual: 95.89%)
For 90% of calls within 30 seconds: 3 agents (actual: 95.21%)

Balancing utilization and service level:
1 agents: 100.00% utilization, 0.68% calls within 60s
2 agents: 61.85% utilization, 69.86% calls within 60s
3 agents: 41.24% utilization, 95.89% calls within 60s
4 agents: 30.93% utilization, 99.32% calls within 60s
5 agents: 24.74% utilization, 100.00% calls within 60s


In [5]:
# Create directory for saving results if it doesn't exist
if not os.path.exists('ImageStep5'):
    os.makedirs('ImageStep5')

# Step 5: Plot Top 5 Key Insights
print("\nStep 5: Plot Top 5 Key Insights")

# We need to simulate by hour first since hourly_results is not defined
# Let's create it using the existing data

# Group data by hour
hourly_data = {}
for hour in range(24):
    hour_df = day_data[day_data['call_started'].dt.hour == hour]
    if len(hour_df) > 0:
        hourly_data[hour] = hour_df

# Initialize hourly_results
hourly_results = {}

# Simulate for each hour
for hour, hour_df in hourly_data.items():
    hourly_results[hour] = {}
    
    # Convert timestamps to seconds from start of hour
    if len(hour_df) > 0:
        hour_start = hour_df['call_started'].min()
        hour_df['arrival_seconds'] = (hour_df['call_started'] - hour_start).dt.total_seconds()
        
        # Run simulation with different numbers of agents
        for num_agents in range(1, 6):
            wait_times, queue_lengths, utilization = simulate_queue(
                hour_df['arrival_seconds'].values,
                hour_df['service_length'].values,
                num_agents
            )
            
            # Calculate service level for 60 seconds
            service_level_60s = sum(1 for w in wait_times if w <= 60) / len(wait_times) * 100 if len(wait_times) > 0 else 100
            
            hourly_results[hour][num_agents] = {
                'wait_times': wait_times,
                'avg_wait': np.mean(wait_times) if len(wait_times) > 0 else 0,
                'service_level_60s': service_level_60s,
                'utilization': utilization
            }

# 1. Optimal Agent Staffing by Hour (most actionable for operations)
# Create a staffing schedule based on 80% service level
optimal_agents = []
for hour in range(24):
    if hour in hourly_results:
        # Find minimum agents for 80% service level
        for num_agents in range(1, 6):
            if num_agents in hourly_results[hour] and hourly_results[hour][num_agents]['service_level_60s'] >= 80:
                optimal_agents.append((hour, num_agents))
                break
        else:
            optimal_agents.append((hour, 5))  # Max tested
    else:
        optimal_agents.append((hour, 1))  # Minimum staffing for quiet hours

# Plot staffing schedule
plt.figure(figsize=(14, 6))
hours, agents = zip(*optimal_agents)
plt.bar(hours, agents, color='teal')
plt.title('Optimal Agent Staffing by Hour (80% Service Level Target)')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Agents')
plt.xticks(range(24))
plt.grid(True, axis='y')
plt.savefig('ImageStep5/optimal_staffing_schedule.png')
plt.close()

# 2. Call Volume Heatmap (critical for understanding patterns)
# Group data by date to analyze across multiple days if available
all_dates_hourly = df.copy()
all_dates_hourly['hour'] = all_dates_hourly['call_started'].dt.hour
call_volume_by_hour_date = all_dates_hourly.groupby(['date', 'hour']).size().reset_index(name='call_count')

# Create heatmap of call volume by date and hour
pivot_calls = call_volume_by_hour_date.pivot(index='date', columns='hour', values='call_count')
plt.figure(figsize=(14, 8))
sns.heatmap(pivot_calls, cmap='YlGnBu', annot=True, fmt='g')
plt.title('Call Volume Heatmap by Date and Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Date')
plt.savefig('ImageStep5/call_volume_heatmap.png')
plt.close()

# 3. Service Level Achievement Analysis (key performance metric)
# Calculate percentage of calls meeting standard by hour
standard_by_hour = df.groupby('hour')['meets_standard'].mean() * 100
plt.figure(figsize=(12, 6))
plt.bar(standard_by_hour.index, standard_by_hour.values, color='green')
plt.axhline(y=95, color='r', linestyle='--', label='95% Target')
plt.title('Percentage of Calls Meeting Standard by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Percentage Meeting Standard')
plt.ylim(0, 100)
plt.grid(True, axis='y')
plt.legend()
plt.xticks(range(24))
plt.savefig('ImageStep5/standard_met_by_hour.png')
plt.close()

# 4. ROI Analysis (business impact)
# Set up assumptions
hourly_agent_cost = 20  # Cost per hour per agent
avg_revenue_per_call = 50  # Average revenue per call
lost_call_threshold = 300  # Seconds after which a call is considered "lost"

# Calculate for each staffing level
costs = []
revenues = []
profits = []

for num_agents in agent_counts:
    # Agent cost
    agent_cost = num_agents * hourly_agent_cost
    
    # Check if 'wait_times' key exists
    if 'wait_times' in results[num_agents]:
        # Lost calls (wait time > threshold)
        lost_calls = sum(1 for w in results[num_agents]['wait_times'] if w > lost_call_threshold)
        retention_rate = 1 - (lost_calls / len(results[num_agents]['wait_times']))
        
        # Estimated revenue
        potential_calls = len(results[num_agents]['wait_times'])
        actual_served = potential_calls * retention_rate
        revenue = actual_served * avg_revenue_per_call
    else:
        # Fallback if wait_times not available
        retention_rate = 0.7  # Assume 70% retention as fallback
        potential_calls = 100  # Assume 100 calls as fallback
        actual_served = potential_calls * retention_rate
        revenue = actual_served * avg_revenue_per_call
    
    # Profit
    profit = revenue - agent_cost
    
    costs.append(agent_cost)
    revenues.append(revenue)
    profits.append(profit)

# Plot ROI analysis
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(agent_counts, profits, color='green', alpha=0.7, label='Profit')
ax1.set_xlabel('Number of Agents')
ax1.set_ylabel('Profit ($)', color='green')
ax1.tick_params(axis='y', labelcolor='green')

ax2 = ax1.twinx()
ax2.plot(agent_counts, [r/c if c > 0 else 0 for r, c in zip(revenues, costs)], 
         color='blue', marker='o', label='ROI')
ax2.set_ylabel('Return on Investment (Revenue/Cost)', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

plt.title('ROI Analysis by Staffing Level')
fig.tight_layout()
plt.savefig('ImageStep5/roi_analysis.png')
plt.close()

# 5. Average Wait Time by Hour of Day (customer experience metric)
plt.figure(figsize=(12, 6))
day_data['hour_of_day'] = day_data['call_started'].dt.hour
avg_wait_by_hour = day_data.groupby('hour_of_day')['wait_length'].mean()
plt.bar(avg_wait_by_hour.index, avg_wait_by_hour.values)
plt.title('Average Wait Time by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Wait Time (seconds)')
plt.grid(True, axis='y')
plt.xticks(range(24))
plt.tight_layout()
plt.savefig('ImageStep5/avg_wait_by_hour.png')
plt.close()

# Print summary of findings
print("\nSummary of Top 5 Key Findings:")
print("-" * 40)
print(f"1. Peak call volume hour: {df['hour'].value_counts().idxmax()}")
print(f"2. Service level achievement: {(df['meets_standard'].mean()*100):.2f}% of calls meet standard")
print(f"3. Average wait time: {df['wait_length'].mean():.2f} seconds")
print(f"4. Most profitable staffing level (estimate): {agent_counts[profits.index(max(profits))]} agents")
print(f"5. Hours requiring maximum staffing: {sum(1 for _, a in optimal_agents if a == 5)}")
print("-" * 40)


Step 5: Plot Top 5 Key Insights


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9564\3767620017.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_df['arrival_seconds'] = (hour_df['call_started'] - hour_start).dt.total_seconds()



Summary of Top 5 Key Findings:
----------------------------------------
1. Peak call volume hour: 8
2. Service level achievement: 91.83% of calls meet standard
3. Average wait time: 17.03 seconds
4. Most profitable staffing level (estimate): 3 agents
5. Hours requiring maximum staffing: 0
----------------------------------------
